In [1]:
import numpy as np
from datetime import datetime
import cv2 as cv2
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import os
import random

d:\python\venv(data_analysis)\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\python\venv(data_analysis)\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
d:\python\venv(data_analysis)\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def get_imgs(number):
    """
    获取图片，并划分训练集
    Parameters:
        number:测试集个数
    Returns:
        test_imgs:测试集
        test_labels:测试集标签
        train_imgs:训练集
        train_labels:训练集标签
    """
    # 读取图片
    imgs = os.listdir('./verify/')
    # 打乱图片顺序
    random.shuffle(imgs)
    # 数据集总共个数
    imgs_num = len(imgs)
    # 按照比例求出测试集个数
    # 测试集，测试数据的路径
    train_imgs = imgs[:number]
    # 根据文件名获取测试集标签
    train_labels = list(map(lambda x: x.split('.')[0], train_imgs))
    return train_imgs, train_labels

In [3]:
get_imgs(10)

(['kbt1.jpg',
  'btz7.jpg',
  '9esu.jpg',
  'a7un.jpg',
  'oids.jpg',
  'mfc1.jpg',
  'g8tc.jpg',
  'pn09.jpg',
  'rtij.jpg',
  '84qs.jpg'],
 ['kbt1',
  'btz7',
  '9esu',
  'a7un',
  'oids',
  'mfc1',
  'g8tc',
  'pn09',
  'rtij',
  '84qs'])

In [4]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
LOW_CASE = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
UP_CASE = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
           'V', 'W', 'X', 'Y', 'Z','_']

CAPTCHA_LIST = NUMBER+LOW_CASE+UP_CASE
CAPTCHA_LEN = 4         # 验证码长度
CAPTCHA_HEIGHT = 30     # 验证码高度
CAPTCHA_WIDTH = 100     # 验证码宽度


# -*- coding:utf-8 -*-
# name: util.py





def text2vec(text, captcha_len=CAPTCHA_LEN, captcha_list=CAPTCHA_LIST):
    """
    验证码文本转为向量
    :param text:
    :param captcha_len:
    :param captcha_list:
    :return: vector 文本对应的向量形式
    """
    text_len = len(text)    # 欲生成验证码的字符长度
    if text_len > captcha_len:
        raise ValueError('验证码最长4个字符')
    vector = np.zeros(captcha_len * len(captcha_list))      # 生成一个一维向量 验证码长度*字符列表长度
    for i in range(text_len):
        vector[captcha_list.index(text[i])+i*len(captcha_list)] = 1     # 找到字符对应在字符列表中的下标值+字符列表长度*i 的 一维向量 赋值为 1
    return vector


def vec2text(vec, captcha_list=CAPTCHA_LIST, captcha_len=CAPTCHA_LEN):
    """
    验证码向量转为文本
    :param vec:
    :param captcha_list:
    :param captcha_len:
    :return: 向量的字符串形式
    """
    vec_idx = vec
    text_list = [captcha_list[int(v)] for v in vec_idx]
    return ''.join(text_list)



def get_next_batch(batch_count=200, width=CAPTCHA_WIDTH, height=CAPTCHA_HEIGHT):
    """
    获取训练图片组
    :param batch_count: default 60
    :param width: 验证码宽度
    :param height: 验证码高度
    :return: batch_x, batch_yc
    """
    data_path = './verify/' # 数据的路径
    batch_x = np.zeros([batch_count, width * height])
    batch_y = np.zeros([batch_count, CAPTCHA_LEN * len(CAPTCHA_LIST)])        
    images,texts= get_imgs(batch_count) # 生成对应的训练集
    for i in range(batch_count):
        image = np.mean(cv2.imread(data_path + images[i]), axis = -1)

        # 将图片数组一维化 同时将文本也对应在两个二维组的同一行
        batch_x[i, :] = image.flatten() / 255
        batch_y[i, :] = text2vec(texts[i])  # 验证码文本的向量形式
    # 返回该训练批次
    return batch_x, batch_y

In [5]:
get_next_batch(batch_count=200)

(array([[0.95816993, 0.95163399, 0.94771242, ..., 0.49019608, 0.48366013,
         0.47973856],
        [0.9503268 , 0.94248366, 0.93856209, ..., 0.48627451, 0.48104575,
         0.47712418],
        [0.93986928, 0.93202614, 0.92810458, ..., 0.48104575, 0.47712418,
         0.47320261],
        ...,
        [0.82222222, 0.81699346, 0.8130719 , ..., 0.46143791, 0.45751634,
         0.45359477],
        [0.89803922, 0.89281046, 0.8875817 , ..., 0.47843137, 0.4745098 ,
         0.46928105],
        [0.86797386, 0.8627451 , 0.85882353, ..., 0.52418301, 0.52026144,
         0.51633987]]),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [6]:
def weight_variable(shape, w_alpha=0.01):
    """
    初始化权值
    :param shape:
    :param w_alpha:
    :return:
   """
    initial = w_alpha * tf.random.normal(shape,dtype=tf.float64)
    return tf.Variable(initial,dtype=tf.float64)


def bias_variable(shape, b_alpha=0.1):
    """
    初始化偏置项
    :param shape:
    :param b_alpha:
    :return:
    """
    initial = b_alpha * tf.random.normal(shape,dtype=tf.float64)
    return tf.Variable(initial,dtype=tf.float64)


def conv2d(x, w):
    """
    卷基层 ：局部变量线性组合，步长为1，模式‘SAME’代表卷积后图片尺寸不变，即零边距
    :param x:
    :param w:
    :return:
    """
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """
    池化层：max pooling,取出区域内最大值为代表特征， 2x2 的pool，图片尺寸变为1/2
    :param x:
    :return:
    """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def cnn_graph(x, keep_prob, size, captcha_list=CAPTCHA_LIST, captcha_len=CAPTCHA_LEN):
    """
    三层卷积神经网络
    :param x:   训练集 image x
    :param keep_prob:   神经元利用率
    :param size:        大小 (高,宽)
    :param captcha_list:
    :param captcha_len:
    :return: y_conv
    """
    # 需要将图片reshape为4维向量
    image_height, image_width = size
    x_image = tf.reshape(x, shape=[-1, image_height, image_width, 1])

    # 第一层
    # filter定义为3x3x1， 输出32个特征, 即32个filter
    w_conv1 = weight_variable([3, 3, 1, 32])    # 3*3的采样窗口，32个（通道）卷积核从1个平面抽取特征得到32个特征平面
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)    # rulu激活函数
    h_pool1 = max_pool_2x2(h_conv1)     # 池化
    h_drop1 = tf.nn.dropout(h_pool1, keep_prob)      # dropout防止过拟合

    # 第二层
    w_conv2 = weight_variable([3, 3, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_drop1, w_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    h_drop2 = tf.nn.dropout(h_pool2, keep_prob)

    # 第三层
    w_conv3 = weight_variable([3, 3, 64, 64])
    b_conv3 = bias_variable([64])
    h_conv3 = tf.nn.relu(conv2d(h_drop2, w_conv3) + b_conv3)
    h_pool3 = max_pool_2x2(h_conv3)
    h_drop3 = tf.nn.dropout(h_pool3, keep_prob)

    """
    原始：30*100图片 第一次卷积后 30*100 第一池化后 15*40
    第二次卷积后 15*40 ，第二次池化后 7.5*20 = > 向下取整 7*20
    第三次卷积后 7*20 ，第三次池化后 3.5*10 = > 向下取整 3*10
    经过上面操作后得到3*10的平面
    """

    # 全连接层
    image_height = int(h_drop3.shape[1])
    image_width = int(h_drop3.shape[2])
    w_fc = weight_variable([image_height*image_width*64, 1024])     # 上一层有64个神经元 全连接层有1024个神经元
    b_fc = bias_variable([1024])
    h_drop3_re = tf.reshape(h_drop3, [-1, image_height*image_width*64])
    h_fc = tf.nn.relu(tf.matmul(h_drop3_re, w_fc) + b_fc)
    h_drop_fc = tf.nn.dropout(h_fc, keep_prob)

    # 输出层
    w_out = weight_variable([1024, len(captcha_list)*captcha_len])
    b_out = bias_variable([len(captcha_list)*captcha_len])
    y_conv = tf.matmul(h_drop_fc, w_out) + b_out
    return y_conv




def accuracy_graph(y, y_conv, width=len(CAPTCHA_LIST), height=CAPTCHA_LEN):
    """
    偏差计算图，正确值和预测值，计算准确度
    :param y: 正确值 标签
    :param y_conv:  预测值
    :param width:   验证码预备字符列表长度
    :param height:  验证码的大小，默认为4
    :return:    正确率
    """
    # 这里区分了大小写 实际上验证码一般不区分大小写,有四个值，不同于手写体识别
    # 预测值
    predict = tf.reshape(y_conv, [-1, height, width])   #
    max_predict_idx = tf.argmax(predict, 2)
    # 标签
    label = tf.reshape(y, [-1, height, width])
    max_label_idx = tf.argmax(label, 2)
    correct_p = tf.equal(max_predict_idx, max_label_idx)    # 判断是否相等
    accuracy = tf.reduce_mean(tf.cast(correct_p, tf.float64))
    return accuracy


def train(height=CAPTCHA_HEIGHT, width=CAPTCHA_WIDTH, y_size=len(CAPTCHA_LIST)*CAPTCHA_LEN):
    """
    cnn训练
    :param height: 验证码高度
    :param width:   验证码宽度
    :param y_size:  验证码预备字符列表长度*验证码长度（默认为4）
    :return:
    """
    # cnn在图像大小是2的倍数时性能最高, 如果图像大小不是2的倍数，可以在图像边缘补无用像素
    # 在图像上补2行，下补3行，左补2行，右补2行
    # np.pad(image,((2,3),(2,2)), 'constant', constant_values=(255,))

    acc_rate = 0.95     # 预设模型准确率标准
    # 按照图片大小申请占位符
    x = tf.compat.v1.placeholder(tf.float64, [None, height * width])
    y = tf.compat.v1.placeholder(tf.float64, [None, y_size])
    # 防止过拟合 训练时启用 测试时不启用 神经元使用率
    keep_prob = tf.compat.v1.placeholder(tf.float64)
    # cnn模型
    y_conv = cnn_graph(x, keep_prob, (height, width))
    # 优化    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_conv))
    # 最小化loss优化 AdaminOptimizer优化
    optimizer = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(loss)
    
    # 计算准确率
    accuracy = accuracy_graph(y, y_conv)
    # 启动会话.开始训练
    saver = tf.compat.v1.train.Saver()
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    config_ = tf.compat.v1.ConfigProto(gpu_options=gpu_options,log_device_placement=True, allow_soft_placement=True)

    with tf.compat.v1.Session(config=config_) as sess:
        sess.run(tf.compat.v1.global_variables_initializer())     # 初始化
        step = 0    # 步数
        while True:
            batch_x, batch_y = get_next_batch(200)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.75})
            print(datetime.now().strftime('%c'), ' step:', step)

            # 每训练一百次测试一次
            if step % 1000 == 0:
                batch_x_test, batch_y_test = get_next_batch(100)
                acc = sess.run(accuracy, feed_dict={x: batch_x_test, y: batch_y_test, keep_prob: 1.0})
                print('---------->',datetime.now().strftime('%c'), ' step:', step, ' accuracy:', acc)
                # 准确率满足要求，保存模型
                if acc > acc_rate:
                    model_path = "./captcha2/model"
                    saver.save(sess, model_path, global_step=step)
                    acc_rate += 0.01
                    if acc_rate > 0.99:     # 准确率达到99%则退出
                        break
            step += 1




In [7]:
train()

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5

Sat Aug 14 11:42:27 2021  step: 0
----------> Sat Aug 14 11:42:27 2021  step: 0  accuracy: 0.0175
Sat Aug 14 11:42:28 2021  step: 1
Sat Aug 14 11:42:28 2021  step: 2
Sat Aug 14 11:42:29 2021  step: 3
Sat Aug 14 11:42:29 2021  step: 4
Sat Aug 14 11:42:29 2021  step: 5
Sat Aug 14 11:42:30 2021  step: 6
Sat Aug 14 11:42:30 2021  step: 7
Sat Aug 14 11:42:31 2021  step: 8
Sat Aug 14 11:42:31 2021  step: 9
Sat Aug 14 11:42:32 2021  step: 10
Sat Aug 14 11:42:32 2021  step: 11
Sat Aug 14 11:42:32 2021  step: 12
Sat Aug 14 11:42:33 2021  step: 13
Sat Aug 14 11:42:33 2021  step: 14
Sat Aug 14 11:42:34 2021  step: 15
Sat Aug 14 11:42:34 2021  step: 16
Sat Aug 14 11:42:35 2021  step: 17
Sat Aug 14 11:42:35 2021  step: 18
Sat Aug 14 11:42:35 2021  step: 19
Sat Aug 14 11:42:36 2021  step: 20
Sat Aug 14 11:42:36 2021  step: 21
Sat Aug 14 11:42

MemoryError: Unable to allocate 4.58 MiB for an array with shape (200, 3000) and data type float64